In [1]:
import requests          # 需要下载安装 
import json              # 内置第三方库
from lxml import etree   # 解析HTML
import json              # to load json

### 构造url

In [2]:
# 构建完整的url 链接
def get_url(start_years, end_years, areaId, areaType, month=1):
    params = {
        'areaInfo[areaId]': areaId,        # 57083
        'areaInfo[areaType]': areaType,    # 2
        'date[year]': start_years,         # 2019
        'date[month]': month               # 8
    }
    # 1 年的数据
    for year in range(start_years, end_years):
        params['date[year]'] = year
        for month in range(1,13):
            params['date[month]'] = month
            main_url = 'http://tianqi.2345.com/Pc/GetHistory'
            get_json(main_url, params)

### 爬取网页信息

In [3]:
# 网页返回的是json对象
def get_json(url, params):
    # define headers
    headers = {
        'User-Agent':
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36',
        'Referer': 'http://tianqi.2345.com/wea_history/57083.htm'
    }
    response = requests.get(url, params=params, headers=headers)
    # response.text or content.decode('utf-8') is not true, because it is not html.
    # so we use the lib json.
    print('爬取 {} 状态是: {}'.format(response.url, response.status_code))
    data = json.loads(response.content.decode('utf-8'))
    get_info(data['data'])

### 获取数据

In [4]:
# 提取数据
days = {
    'date':[],
    'week':[],
    'max_temperature':[],
    'min_temperature':[],
    'day_status':[],
    'wind':[],
    'aqi':[],
    'aqi_status':[]
}
def get_info(data):
    # html = etree.HTML(html_elements)  # 初始化生成一个XPath解析对象
    # result = etree.tostring(html, encoding='utf-8')  # 解析对象输出代码
    html=etree.HTML(data,etree.HTMLParser())
    trs = html.xpath('//tr')[1:]
    for tr in trs:
        data = tr.xpath('.//text()')
        # 消去垃圾值
        data = [x for x in data if x != '\n        ' and x != '    ']
        # 划分数据
        res_data = []
        for x in data:
            res_data += x.split()
        # print(res_data)


        # 添加数据
        days['date'].append(res_data[0])
        days['week'].append(res_data[1])
        days['max_temperature'].append(res_data[2])
        days['min_temperature'].append(res_data[3])
        days['day_status'].append(res_data[4])
        days['wind'].append(res_data[5])
        days['aqi'].append(res_data[6])
        days['aqi_status'].append(res_data[7])

In [5]:
if __name__ == '__main__':
    get_url(2017, 2020, 57083, 2)

爬取 http://tianqi.2345.com/Pc/GetHistory?areaInfo%5BareaId%5D=57083&areaInfo%5BareaType%5D=2&date%5Byear%5D=2017&date%5Bmonth%5D=1 状态是: 200
爬取 http://tianqi.2345.com/Pc/GetHistory?areaInfo%5BareaId%5D=57083&areaInfo%5BareaType%5D=2&date%5Byear%5D=2017&date%5Bmonth%5D=2 状态是: 200
爬取 http://tianqi.2345.com/Pc/GetHistory?areaInfo%5BareaId%5D=57083&areaInfo%5BareaType%5D=2&date%5Byear%5D=2017&date%5Bmonth%5D=3 状态是: 200
爬取 http://tianqi.2345.com/Pc/GetHistory?areaInfo%5BareaId%5D=57083&areaInfo%5BareaType%5D=2&date%5Byear%5D=2017&date%5Bmonth%5D=4 状态是: 200
爬取 http://tianqi.2345.com/Pc/GetHistory?areaInfo%5BareaId%5D=57083&areaInfo%5BareaType%5D=2&date%5Byear%5D=2017&date%5Bmonth%5D=5 状态是: 200
爬取 http://tianqi.2345.com/Pc/GetHistory?areaInfo%5BareaId%5D=57083&areaInfo%5BareaType%5D=2&date%5Byear%5D=2017&date%5Bmonth%5D=6 状态是: 200
爬取 http://tianqi.2345.com/Pc/GetHistory?areaInfo%5BareaId%5D=57083&areaInfo%5BareaType%5D=2&date%5Byear%5D=2017&date%5Bmonth%5D=7 状态是: 200
爬取 http://tianqi.2345.com/P

In [6]:
import pandas  as pd

In [7]:
df =  pd.DataFrame(days, index=days['date'])

In [8]:
df.head()

,date,week,max_temperature,min_temperature,day_status,wind,aqi,aqi_status
2017-01-01,2017-01-01,周日,-,-,多云,无持续风向微风,372,严重
2017-01-02,2017-01-02,周一,-,-,霾,无持续风向微风,361,严重
2017-01-03,2017-01-03,周二,-,-,霾~雾,无持续风向微风,280,重度
2017-01-04,2017-01-04,周三,9°,2°,小雨,无持续风向微风,193,中度
2017-01-05,2017-01-05,周四,5°,1°,小雨,无持续风向微风,216,重度


In [9]:
df.to_excel('result.xlsx', index=None)